In [ ]:
include("_.jl")

In [ ]:
for (ro, di_, fi_) in walkdir(PAI)

    for fi in fi_

        pa = joinpath(ro, fi)

        ti, ex = splitext(fi)

        if ex in [".gmt", ".tsv"]

            DictExtension.write(joinpath(PAO, string(ti, ".json")), GMTAccess.read(pa))

        end

    end

end